In [1]:
!pip install transformers
#update scikit-learn
!pip uninstall scikit-learn -y
!pip install scikit-learn
from transformers import pipeline

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 30.8 MB/s 
     |████████████████████████████████| 596 kB 30.2 MB/s 
     |████████████████████████████████| 59 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 28.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Found existing installation: scikit-learn 0.22.2.post1
Uninstalling scikit-learn-0.22.2.post1:
  Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 23.2 MB 47.3 MB/s 


In [ ]:
# this classifier is not working well at all, returns same probabilities for all classes.
#classifier = pipeline("zero-shot-classification",
                      #model="gsarti/scibert-nli")

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at gsarti/scibert-nli and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/135 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/217k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [2]:
# get biomimicry papers
import pandas as pd
url = 'https://raw.githubusercontent.com/nasa-petal/petal-labeler-data-pipeline/master/FinalFile/golden.json'
df = pd.read_json(url, orient='columns')
df.head(10)

,paper,mag,venue_mag,author,reference,title,abstract,petalID,doi,venue,level1,level2,level3,url,isBiomimicry,fullDocLink,isOpenAccess
0,2103410568,"['bubble nest', 'nest', 'mixing', 'bubble', 'p...",['Biology Letters'],"[2346835213, 2098042950]","[2130285640, 2066345165, 2054319467, 204771406...","['building', 'home', 'foam', 'tungara', 'frog'...","['frogs', 'build', 'foam', 'nests', 'floating'...",0,10.1098/RSBL.2009.0934,"[""Weird Nature: An Astonishing Exploration of ...","['physically_assemble/disassemble', 'protect_f...","['physically_assemble_structure', 'protect_fro...","['protect_from_animals', 'protect_from_loss_of...",https://royalsocietypublishing.org/doi/10.1098...,Y,https://royalsocietypublishing.org/doi/10.1098...,True
1,2138292607,"['sunset', 'earth s magnetic field', 'compass'...",['Proceedings of the National Academy of Scien...,"[2132083079, 2425702268, 2552946098]","[1493129647, 2037761037, 1984592609, 213699427...","['nocturnal', 'mammal', 'greater', 'mouse', 'e...","['evidence', 'suggests', 'bats', 'detect', 'ge...",1,10.1073/PNAS.0912477107,['Proceedings of the National Academy of Scien...,['sense_send_or_process_information'],['sense_signals/environmental_cues'],['sense_spatial_awareness/balance/orientation'],https://www.pnas.org/content/107/15/6941,Y,https://www.pnas.org/content/107/15/6941.full.pdf,True
2,2005539166,"['sepia mestus', 'optomotor response', 'cuttle...",['The Journal of Experimental Biology'],"[2163942483, 3088803717]","[2035108601, 2155571491, 2159857711, 207521876...","['polarization', 'sensitivity', 'two', 'specie...","['existence', 'polarization', 'sensitivity', '...",2,10.1242/JEB.042937,"['The Journal of Experimental Biology', 'Curre...",['sense_send_or_process_information'],['sense_signals/environmental_cues'],"['sense_light_in_the_non-visible_spectrum', 's...",https://jeb.biologists.org/content/213/19/3364,Y,https://journals.biologists.com/jeb/article-pd...,True
3,2151557512,"['energy source', 'cellulase', 'archaea', 'cel...",['Nature Communications'],"[3146712807, 3146712807, 2189911855, 265906908...","[2141885858, 2120772351, 2108929776, 212024499...","['identification', 'characterization', 'multid...","['archaea', 'microorganisms', 'wide', 'range',...",3,10.1038/NCOMMS1373,['Nature Communications'],"['chemically_modify_or_change_energy_state', '...","['chemically_break_down', 'protect_from_non-li...","['chemically_break_down_organic_compounds', 'p...",https://www.nature.com/articles/ncomms1373,Y,https://www.nature.com/articles/ncomms1373.pdf,True
4,2160542693,"['galactan', 'coralline algae', 'cell wall', '...",['Journal of Phycology'],"[1790079306, 2557664461, 189902495, 2140516860]","[2001396309, 2057007315, 1968834637, 212515227...","['differences', 'polysaccharide', 'structure',...","['articulated', 'coralline', 'calliarthron', '...",4,10.1111/J.1529-8817.2010.00828.X,"['The Journal of Experimental Biology', 'Natur...",['maintain_structural_integrity'],['manage_structural_forces'],"['manage_shear', 'manage_stress/strain']",https://onlinelibrary.wiley.com/doi/abs/10.111...,Y,,False
5,2130285640,"['nest', 'peptide sequence', 'insect', 'cystat...",['Proceedings of The Royal Society B: Biologic...,"[2140984283, 2490734702, 2116739833, 2098042950]","[1975304761, 2089047063, 2034767282, 210406370...","['foam', 'nest', 'components', 'tungara', 'fro...","['foam', 'nests', 'tungara', 'frog', 'engystom...",5,10.1098/RSPB.2008.1939,"[""Weird Nature: An Astonishing Exploration of ...","['maintain_structural_integrity', 'protect_fro...","['manage_structural_forces', 'protect_from_liv...","['manage_impact', 'protect_from_animals', 'pro...",https://royalsocietypublishing.org/doi/10.1098...,Y,https://royalsocietypublishing.org/doi/10.1098...,True
6,2168114966,"['eremobatidae', 'suctorial', 'eremobates', 's...",['Arthropod Structure & Development'],"[2907238235, 2075816131, 1452472624, 215895902...","[1676421722, 2102501940, 2242001249, 209572581...","['suctorial', 'organ', 'solifugae', 'ara

In [3]:
# remap level1 labels
'''
df.level1 = df.level1.replace({'physically_assemble/disassemble' : 'physically assembling or disassembling',
                               'protect_from_harm':'protecting',
                               'sense_send_or_process_information':'sensing, sending or processing information',
                               'sense,_send,_or_process_information': 'sensing, sending or processing information',
                               'chemically_modify_or_change_energy_state':'chemically assembling or breaking down',
                               'maintain_structural_integrity':'managing mechanical forces',
                               'attach':'attaching',
                               'move':'movement',
                               'process_resources':'manipulating solids, liquids, gases or energy',
                               'sustain_ecological_community':'sustaining ecological community',
                               'maintain_ecological_community':'sustaining ecological community',
                               'manipulate_solids,_liquids,_gases,_or_energy':'manipulating solids, liquids, gases or energy',
                               'change_size_or_color':''}, regex=True)
'''
df.level1 = df.level1.replace({'physically_assemble/disassemble' : 'physically_assemble_or_disassemble',
                               'sense,_send,_or_process_information': 'sense_send_or_process_information',
                               'maintain_ecological_community':'sustain_ecological_community',
                               'manipulate_solids,_liquids,_gases,_or_energy':'manipulate_solids_liquids_gases_or_energy'}, regex=True)

In [4]:
# remove rows with no level1 labels, no paper ids, or no titles
df = df[df['level1'] != '[]']
df = df[df['level1'] != "['']"]
df = df[df['paper'] != '']
df = df[df['title'] != '']
df = df[df['title'] != '[]']
df = df[df['title'] != "['']"]
df

,paper,mag,venue_mag,author,reference,title,abstract,petalID,doi,venue,level1,level2,level3,url,isBiomimicry,fullDocLink,isOpenAccess
0,2103410568,"['bubble nest', 'nest', 'mixing', 'bubble', 'p...",['Biology Letters'],"[2346835213, 2098042950]","[2130285640, 2066345165, 2054319467, 204771406...","['building', 'home', 'foam', 'tungara', 'frog'...","['frogs', 'build', 'foam', 'nests', 'floating'...",0,10.1098/RSBL.2009.0934,"[""Weird Nature: An Astonishing Exploration of ...","['physically_assemble_or_disassemble', 'protec...","['physically_assemble_structure', 'protect_fro...","['protect_from_animals', 'protect_from_loss_of...",https://royalsocietypublishing.org/doi/10.1098...,Y,https://royalsocietypublishing.org/doi/10.1098...,True
1,2138292607,"['sunset', 'earth s magnetic field', 'compass'...",['Proceedings of the National Academy of Scien...,"[2132083079, 2425702268, 2552946098]","[1493129647, 2037761037, 1984592609, 213699427...","['nocturnal', 'mammal', 'greater', 'mouse', 'e...","['evidence', 'suggests', 'bats', 'detect', 'ge...",1,10.1073/PNAS.0912477107,['Proceedings of the National Academy of Scien...,['sense_send_or_process_information'],['sense_signals/environmental_cues'],['sense_spatial_awareness/balance/orientation'],https://www.pnas.org/content/107/15/6941,Y,https://www.pnas.org/content/107/15/6941.full.pdf,True
2,2005539166,"['sepia mestus', 'optomotor response', 'cuttle...",['The Journal of Experimental Biology'],"[2163942483, 3088803717]","[2035108601, 2155571491, 2159857711, 207521876...","['polarization', 'sensitivity', 'two', 'specie...","['existence', 'polarization', 'sensitivity', '...",2,10.1242/JEB.042937,"['The Journal of Experimental Biology', 'Curre...",['sense_send_or_process_information'],['sense_signals/environmental_cues'],"['sense_light_in_the_non-visible_spectrum', 's...",https://jeb.biologists.org/content/213/19/3364,Y,https://journals.biologists.com/jeb/article-pd...,True
3,2151557512,"['energy source', 'cellulase', 'archaea', 'cel...",['Nature Communications'],"[3146712807, 3146712807, 2189911855, 265906908...","[2141885858, 2120772351, 2108929776, 212024499...","['identification', 'characterization', 'multid...","['archaea', 'microorganisms', 'wide', 'range',...",3,10.1038/NCOMMS1373,['Nature Communications'],"['chemically_modify_or_change_energy_state', '...","['chemically_break_down', 'protect_from_non-li...","['chemically_break_down_organic_compounds', 'p...",https://www.nature.com/articles/ncomms1373,Y,https://www.nature.com/articles/ncomms1373.pdf,True
4,2160542693,"['galactan', 'coralline algae', 'cell wall', '...",['Journal of Phycology'],"[1790079306, 2557664461, 189902495, 2140516860]","[2001396309, 2057007315, 1968834637, 212515227...","['differences', 'polysaccharide', 'structure',...","['articulated', 'coralline', 'calliarthron', '...",4,10.1111/J.1529-8817.2010.00828.X,"['The Journal of Experimental Biology', 'Natur...",['maintain_structural_integrity'],['manage_structural_forces'],"['manage_shear', 'manage_stress/strain']",https://onlinelibrary.wiley.com/doi/abs/10.111...,Y,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,2145624080,"['contact area', 'functional design', 'interlo...",['Integrative and Comparative Biology'],"[2160081516, 2027173136, 2706417588, 264087278...","[1676421722, 1670765730, 2032621787, 200883865...","['structural', 'design', 'biomechanics', 'fric...","['design', 'attachment', 'devices', 'insects',...",1126,10.1093/ICB/42.6.1127,"['For the Love of Insects', 'Integrative and C...",['attach'],[],[],https://academic.oup.com/icb/article/42/6/1127...,Y,,False
1127,1982097444,"['fascicle', 'electromyography', 'sonomicromet...",['The Journal of Experimental Biology'],"[2250909103, 2096214488, 2977094504, 2091821682]","[2157714820, 1501891458, 184374695, 2005861641...","['dynamics', 'leg', 'muscle', 'function', 'tam...","['goal', 'examine', 'vivo', 'force', 'length',...",1127,10.1242/JEB.00764,"['The Journal of Experimental

In [5]:
# convert level1 array strings to actual arrays
from ast import literal_eval
df['level1'] = df['level1'].apply(literal_eval)
df['title'] = df['title'].apply(literal_eval)
df['abstract'] = df['abstract'].apply(literal_eval)

# show unique level1 labels in the dataset
from itertools import chain

unique_lists_in_items = df.level1
set_of_items = set(chain(*unique_lists_in_items))
print(set_of_items)

{'move', 'chemically_modify_or_change_energy_state', 'maintain_structural_integrity', 'attach', 'process_resources', 'physically_assemble_or_disassemble', 'sustain_ecological_community', 'sense_send_or_process_information', 'change_size_or_color', 'protect_from_harm'}


In [6]:
'''
sequences_to_classify = [{"MAGID":2114147125,"abstract":"Evidence for self-cleaning in gecko setae. A tokay gecko can cling to virtually any surface and support its body mass with a single toe by using the millions of keratinous setae on its toe pads. Each seta branches into hundreds of 200-nm spatulae that make intimate contact with a variety of surface profiles. We showed previously that the combined surface area of billions of spatulae maximizes van der Waals interactions to generate large adhesive and shear forces. Geckos are not known to groom their feet yet retain their stickiness for months between molts. How geckos manage to keep their feet clean while walking about with sticky toes has remained a puzzle until now. Although self-cleaning by water droplets occurs in plant and animal surfaces, no adhesive has been shown to self-clean. In the present study, we demonstrate that gecko setae are a self-cleaning adhesive. Geckos with dirty feet recovered their ability to cling to vertical surfaces after only a few steps. Self-cleaning occurred in arrays of setae isolated from the gecko. Contact mechanical models suggest that self-cleaning occurs by an energetic disequilibrium between the adhesive forces attracting a dirt particle to the substrate and those attracting the same particle to one or more spatulae. We propose that the property of self-cleaning is intrinsic to the setal nanostructure and therefore should be replicable in synthetic adhesive materials in the future."},
                         {"MAGID":2055451617,"abstract":"The plant surface is by default flat, and development away from this default is thought to have some function of evolutionary advantage. Although the functions of many plant epidermal cells have been described, the function of conical epidermal cells, a defining feature of petals in the majority of insect-pollinated flowers, has not. The location and frequency of conical cells have led to speculation that they play a role in attracting animal pollinators. Snapdragon (Antirrhinum) mutants lacking conical cells have been shown to be discriminated against by foraging bumblebees. Here we investigated the extent to which a difference in petal surface structure influences pollinator behavior through touch-based discrimination. To isolate touch-based responses, we used both biomimetic replicas of petal surfaces and isogenic Antirrhinum lines differing only in petal epidermal cell shape. We show that foraging bumblebees are able to discriminate between different surfaces via tactile cues alone. We find that bumblebees use color cues to discriminate against flowers that lack conical cells--but only when flower surfaces are presented at steep angles, making them difficult to manipulate. This facilitation of physical handling is a likely explanation for the prevalence of conical epidermal petal cells in most flowering plants."},
                         {"MAGID":2005631208,"abstract":"In stark contrast to most aggressive predators, Dosidicus gigas (jumbo squids) do not use minerals in their powerful mouthparts known as beaks. Their beaks instead consist of a highly sclerotized chitinous composite with incremental hydration from the tip to the base. We previously reported l-3,4-dihydroxyphenylalanine (dopa)-histidine (dopa-His) as an important covalent cross-link providing mechanical strengthening to the beak material. Here, we present a more complete characterization of the sclerotization chemistry and describe additional cross-links from D. gigas beak. All cross-links presented in this report share common building blocks, a family of di-, tri-, and tetra-histidine-catecholic adducts, that were separated by affinity chromatography and high performance liquid chromatography (HPLC) and identified by tandem mass spectroscopy and proton nuclear magnetic resonance (1H NMR). The data provide additional insights into the unusually high cross-link density found in mature beaks. Furthermore, we propose both a low molecular weight catechol, and peptidyl-dopa, to be sclerotization agents of squid beak. This appears to represent a new strategy for forming hard tissue in animals. The interplay between covalent cross-linking and dehydration on the graded properties of the beaks is discussed."},
                         {"MAGID":156627570,"abstract":"As human activities continue to alter the global nitrogen cycle, the ability to predict the impact of increased nitrogen loading to freshwater systems is becoming more and more important. Nitrogen retention is of particular interest because it is through its combined processes (denitrification, nitrogen sedimentation and uptake by aquatic plants) that local and downstream nitrogen concentrations are reduced. Here, we compare the magnitude of nitrogen retention and its components in wetlands, lakes and rivers. We show that wetlands retain the highest proportion of total nitrogen loading, followed by lakes and then rivers. The differences in the proportion of N retained among systems is explained almost entirely by differences in water discharge. Denitrification is the primary mechanism of nitrogen retention, followed by nitrogen sedimentation and uptake by aquatic plants."},
                         {"MAGID":2076699264,"abstract":"Cooling is a significant end-use of energy globally and a major driver of peak electricity demand. Air conditioning, for example, accounts for nearly fifteen per cent of the primary energy used by buildings in the United States. A passive cooling strategy that cools without any electricity input could therefore have a significant impact on global energy consumption. To achieve cooling one needs to be able to reach and maintain a temperature below that of the ambient air. At night, passive cooling below ambient air temperature has been demonstrated using a technique known as radiative cooling, in which a device exposed to the sky is used to radiate heat to outer space through a transparency window in the atmosphere between 8 and 13¬†micrometres. Peak cooling demand, however, occurs during the daytime. Daytime radiative cooling to a temperature below ambient of a surface under direct sunlight has not been achieved because sky access during the day results in heating of the radiative cooler by the Sun. Here, we experimentally demonstrate radiative cooling to nearly 5¬†degrees Celsius below the ambient air temperature under direct sunlight. Using a thermal photonic approach, we introduce an integrated photonic solar reflector and thermal emitter consisting of seven layers of HfO2 and SiO2 that reflects 97 per cent of incident sunlight while emitting strongly and selectively in the atmospheric transparency window. When exposed to direct sunlight exceeding 850 watts per square metre on a rooftop, the photonic radiative cooler cools to 4.9¬†degrees Celsius below ambient air temperature, and has a cooling power of 40.1 watts per square metre at ambient air temperature. These results demonstrate that a tailored, photonic approach can fundamentally enable new technological possibilities for energy efficiency. Further, the cold darkness of the Universe can be used as a renewable thermodynamic resource, even during the hottest hours of the day."},
                         {"MAGID":1986827150,"abstract":"Autotomy refers to the voluntary shedding of a body part; a renowned example is tail loss among lizards as a response to attempted predation. Although many aspects of lizard tail autotomy have been studied, the detailed morphology and mechanism remains unclear. In the present study, we showed that tail shedding by the Tokay gecko (Gekko gecko) and the associated extracellular matrix (ECM) rupture were independent of proteolysis. Instead, lizard caudal autotomy relied on biological adhesion facilitated by surface microstructures. Results based on bio-imaging techniques demonstrated that the tail of Gekko gecko was pre-severed at distinct sites and that its structural integrity depended on the adhesion between these segments."},
                         {"MAGID":1983825491,"abstract":"Competition among males for access to reproductive opportunities is a central tenet of behavioural biology that has critical implications for studies of mating systems, sexual selection and the evolution of numerous phenotypic traits. Given the expectation that males should compete vigorously for access to females, it may at first seem paradoxical that males in some species cooperate to reproduce, often resulting in the apparent sacrifice of direct fitness by some members of these cooperative partnerships. Because this form of cooperation lies at the interface between natural, sexual and kin selection, studies of the adaptive consequences of male reproductive cooperation may yield important insights into how complex and sometimes conflicting selective pressures shape individual behaviour. Here, we define and review examples of reproductive cooperation among male animals. We take an integrative approach to reviewing the potential causes of maleemale cooperation, including potential adaptive hypotheses, ecological correlates, phylogenetic patterns and physiological mechanisms. The impact of male reproductive cooperation on sexual selection theory is also discussed. We conclude by outlining several important directions for future research, including efforts to improve understanding of the ecological and demographic contexts in which male reproductive cooperation occurs. Collectively, such analyses promise to improve our understanding of multiple fundamental concepts in evolutionary biology. 2013 The Authors. Published on behalf of The Association for the Study of Animal Behaviour by Elsevier Ltd. All rights reserved."}]
'''
candidate_labels = ['Attach', 'Attach permanently', 'Attach temporarily', 'Modify color or camouflage', 'Change structural color', 'Change chemical color or pigmentation', 'Camouflage or mimicry', 'Modify size shape or material properties', 'Change size or shape', 'Change material properties', 'Modify or convert energy', 'Modify or convert electrical energy', 'Modify or convert magnetic energy', 'Modify or convert chemical energy', 'Modify or convert mechanical energy', 'Modify or convert thermal energy', 'Modify or convert light energy', 'Assemble or break down structure', 'Physically assemble structure', 'Break down structure', 'Move on or through solids liquids or gases', 'Passive movement', 'Passively move through or on solids', 'Passively move through or on liquids', 'Passively move through gases', 'Passively move through or on granular media', 'Active movement', 'Actively move through or on solids', 'Actively move through or on liquids', 'Actively move through gases', 'Actively move through or on granular media', 'Protect from living or non-living threats', 'Protect from living threats', 'Protect from animals', 'Protect from plants', 'Protect from fungi', 'Protect from microbes', 'Protect from non-living threats', 'Protect from solids', 'Protect from excess liquids', 'Protect from loss of liquids', 'Protect from gases', 'Protect from wind', 'Protect from temperature', 'Protect from fire', 'Protect from ice', 'Protect from light', 'Protect from chemicals', 'Protect from radiation', 'Manage mechanical forces', 'Manage external forces', 'Manage impact', 'Regulate drag or turbulence', 'Manage stress or strain', 'Prevent buckling', 'Prevent fracture or rupture', 'Manage shear', 'Prevent or allow deformation', 'Prevent degradation', 'Prevent fatigue', 'Control erosion', 'Regulate wear', 'Sustain ecological community', 'Individual benefit', 'Regulate reproduction or growth', 'Self-replicate', 'Maintain homeostasis or equillibrium', 'Group benefit', 'Coordinate by self-organization', 'Cooperate within or between species', 'Compete within or between species', 'Manage environmental disturbances', 'Manage populations or habitats', 'Manage pests or diseases', 'Maintain biodiversity', 'Chemically assemble or break down', 'Chemically assemble', 'Chemically assemble inorganic compounds', 'Chemically assemble organic compounds', 'Self-assemble', 'Chemically break down', 'Chemically break down inorganic compounds', 'Chemically break down organic compounds', 'Sense send or process information', 'Send signals', 'Send light signals in the visible spectrum', 'Send light signals in the non-visible spectrum', 'Send sound signals', 'Send tactile signals', 'Send chemical signals', 'Send vibratory signals', 'Send electrical or magnetic signals', 'Process signals', 'Differentiate signal from noise', 'Convert signals', 'Respond to signals', 'Sense signals or environmental cues', 'Sense light in the visible spectrum', 'Sense light in the non-visible spectrum', 'Sense electricity or magnetism', 'Sense disease in a living system', 'Sense touch and mechanical forces', 'Sense chemicals', 'Sense atmospheric conditions', 'Sense sound or vibrations', 'Sense temperature cues', 'Sense motion', 'Sense spatial awareness or balance or orientation', 'Sense shape and or or pattern', 'Manipulate solids liquids gases or energy', 'Capture resources', 'Capture solids', 'Capture liquids', 'Capture gases', 'Capture energy', 'Absorb and or or filter resources', 'Absorb and or or filter solids', 'Absorb and or or filter liquids', 'Absorb and or or filter gases', 'Store resources', 'Store solids', 'Store liquids', 'Store gases', 'Store energy', 'Distribute or expel resources', 'Distribute or expel solids', 'Distribute or expel liquids', 'Distribute or expel gases', 'Distribute or expel energy', 'Detox or purify']
candidate_labels_leaves = []
candidate_labels_lvl1 = ['physically assembling or disassembling',
                         'protecting',
                         'sensing, sending or processing information',
                         'chemically assembling or breaking down',
                         'managing mechanical forces',
                         'attaching',
                         'movement',
                         'manipulating solids, liquids, gases or energy',
                         'sustaining ecological community']
candidate_labels_lvl1b = ['assembling',
                         'protecting',
                         'signals',
                         'chemical assembly',
                         'structural integrity',
                         'attaching',
                         'movement',
                         'purifying',
                         'ecological community']

#Define the class labels
#this is the list of labels we're currently using
candidate_labels_lvl1c = ['physically_assemble_or_disassemble',
                          'protect_from_harm',
                          'sense_send_or_process_information',
                          'chemically_modify_or_change_energy_state',
                          'maintain_structural_integrity',
                          'attach',
                          'move',
                          'process_resources',
                          'sustain_ecological_community',
                          'change_size_or_color']

In [7]:
# take 5 papers from each level1 label = 50 samples for labelling.
samples = []
sampling = df

for label in candidate_labels_lvl1c:
  docs_with_label = sampling[pd.DataFrame(sampling['level1'].tolist()).isin([label]).any(1).values]
  sample = docs_with_label.sample(n = 3)
  samples.append(sample)
  sampling = sampling.drop(sample.index)
df_samples = pd.concat(samples)

In [8]:
# instantiate our classifier
classifier2 = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
# generate scores for each label for each paper
import torch

col_scores = []
indexes = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

for idx, sequence in df_samples.iterrows():
  title_sentence = " ".join(sequence['title'])
  abs_sentence = " ".join(sequence['abstract'])
  sentence = title_sentence + " " + abs_sentence
  print(sentence)

  doc_scores = classifier2(sentence, candidate_labels_lvl1c, multi_label=True, device=device)
  row = []
  indexes.append(sequence['paper'])
  for label in candidate_labels_lvl1c:
    index = doc_scores['labels'].index(label)
    doc_score = doc_scores['scores'][index]
    row.append(doc_score)
  col_scores.append(row)
  if (idx == 5):
    pass; #break;

df_sys = pd.DataFrame(col_scores, index = indexes, columns = candidate_labels_lvl1c)
df_sys

harnessing disorder onychophorans use highly unstructured proteins silks prey capture onychophora ancient carnivorous soft bodied invertebrates capture prey slime originates dedicated glands located side head biochemical composition unusual nature mechanism ensnaring thread formation remained elusive examined gene expression gland australian onychophoran euperipatoides rowelli matched expressed sequence tags separated proteins revealed three categories protein unique high molecular weight proline rich smaller concentrations lectins small peptides latter two likely act protease inhibitors antimicrobial agents predominant 200 kda composed tandem repeated motifs distinguished unusually charged residue content unlike highly structured silks spiders insects lack ordered secondary structure over entire length propose expulsion onto evaporative water loss triggers glass transition change solution resulting adhesive enmeshing assisted cross linking complementary hydrophobic regions developed e

In [ ]:
# create our ground truth matrix D x T where row indexes are the paper MAGIDS, and the columns are the topics. A 1 means the paper had the label in the ground truth data, a 0 means it didn't.
y_true = []
indexes = []

for idx, sequence in df_samples.iterrows():
  #doc = df.loc[[sequence['MAGID']]]
  row = []
  indexes.append(sequence['paper'])
  for label in candidate_labels_lvl1c:
    #print(doc['level1'])
    if (label in sequence['level1']):
      row.append(1)
    else:
      row.append(0)
  y_true.append(row)
  if (idx == 5):
    pass; #break;
df_truth = pd.DataFrame(y_true, index = indexes, columns = candidate_labels_lvl1c)
df_truth

In [ ]:
# generate Average Precision scores for each topic
import numpy as np
from sklearn.metrics import average_precision_score

y_true = np.array(y_true)
y_scores = np.array(col_scores)

y_true_t = y_true.transpose()
y_scores_t = y_scores.transpose()
ap_by_topic = []

for idx, topic in enumerate(y_true_t):
   #print(topic)
   #print(y_scores[idx])
   ap = average_precision_score(topic, y_scores_t[idx])
   #print(ap)
   ap_by_topic.append(ap)

tmp = [ap_by_topic]
df_ap_by_topic = pd.DataFrame(tmp, columns = candidate_labels_lvl1c)

# Generate MAP and gMAP scores

#map = average_precision_score(y_true, y_scores)
map = sum(ap_by_topic) / len(ap_by_topic)

multiply = 1
values = ap_by_topic
n = len(values)

for i in values:
    multiply = (multiply)*(i)

gmap = (multiply)**(1/n)

print(' MAP Score: ', map)
print('gMAP Score: ', gmap)
print('Average Precision by Class')
df_ap_by_topic